In [2]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
from ydata_profiling import ProfileReport

### Exploratory Data Analysis

In [ ]:
df_songs = pd.read_csv(r'../data/data.csv')
profile = ProfileReport(df_songs, title="Pandas Profiling Report")
profile.to_widgets()

### Data Pre-Processing

In [68]:

# Drop useless columns (High Cardinality)
df_songs = df_songs.drop(['Artist Name', 'Track Name'], axis=1)

# Fill empty values in 'popularity' with the mean
df_songs['Popularity'] = df_songs['Popularity'].fillna(df_songs['Popularity'].mean())
df_songs['key'] = df_songs['key'].fillna(df_songs['key'].mode()[0])
df_songs['instrumentalness'] = df_songs['instrumentalness'].fillna(df_songs['instrumentalness'].mean())

# Normalization
df_norm = df_songs[['Popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms']]
mean_norm = df_norm.mean()
std_norm = df_norm.std()
df_norm = (df_norm-mean_norm) / std_norm

# Categorical Features
df_categorical = df_songs[['key', 'mode', 'time_signature', 'Class']]

# Put everything together
df_preprocessed = df_norm.merge(df_categorical, how='inner', left_index=True, right_index=True)

### Artificial Neural Network Modelling